In [57]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications import EfficientNetB0
from PIL import Image
import albumentations as alb
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

In [61]:
# constants
leaf_df = pd.read_csv('/home/patrick/Projects/cassava_data/train.csv')
leaf_image_directory = '../cassava_data/train_images'

leaf_df['label']= leaf_df['label'].astype(int)

validate_split = 0.2
batch_size = 4

img_width = 224
img_height = 224

input_shape = (img_height, img_width, 3)

learning_rate = 0.001
l2_penalty = 0.0001

dropout_rate = 0.3

In [62]:
#####
# define class weights
#####

class_nums = []
for x in range(5):
    num = leaf_df[leaf_df['label'] == x].count()[0]
    class_nums.append(num)

class_weights = {}
for x in range(5):
    class_weights[x] = max(class_nums)/class_nums[x]

print(class_weights)

{0: 12.1048758049678, 1: 6.010963910461398, 2: 5.514668901927913, 3: 1.0, 4: 5.105937136204889}


In [63]:
#####
# create train and validate dataframes
#####

# establish amounts for train/validate split
num_images = leaf_df.shape[0]
num_valid = int(num_images*validate_split)
num_train = num_images - num_valid

# shuffle data
shuffled_df = leaf_df.sample(frac=1)

# create validation and traning dataframes
valid_df = shuffled_df.iloc[:num_valid,:]
train_df = shuffled_df.iloc[num_valid+1:,:]

valid_x = valid_df['image_id'].tolist()
valid_x = [leaf_image_directory + "/" + element for element in valid_x]
valid_y = valid_df['label'].tolist()
train_x = train_df['image_id'].tolist()
train_x = [leaf_image_directory + "/" + element for element in train_x]
train_y = train_df['label'].tolist()

In [1]:
#####
# define augmentation function
#####
transform = alb.Compose([
                         #alb.RandomCrop(width=img_width, height=img_height, p=1.0),
                         alb.HorizontalFlip(),
                         alb.VerticalFlip(),
                         alb.Rotate(),
                         alb.GaussianBlur(p=0.3),
                         #alb.CoarseDropout(min_holes=1,
                         #                  max_holes=20,
                         #                  min_height=20,
                         #                  max_height=50,
                         #                  min_width=20,
                         #                  max_width=50),
                         alb.RandomBrightnessContrast(),
                         alb.ToGray(p=0.3),
                         #alb.HueSaturationValue(),
                         alb.RandomBrightnessContrast(p=0.3),
                       ])

def augment(img):
    #np_img = np.array(img)
    aug_img = transform(image=img)['image']
    return aug_img

# test the preprocessing function
img = Image.open('../cassava_data/train_images/15982075.jpg')
print(img.size)
##img = img.resize((300,300))

img = img.resize((img_width, img_height))
img = np.array(img)
img = np.float32(img/255.0)
augmented= augment(img)
plt.imshow(img)
plt.show()
plt.imshow(augmented)
plt.show()

NameError: name 'alb' is not defined

In [69]:
#####
# define data generator functions
#####

# training values
train_names = train_df['image_id'].values
train_labels = train_df['label'].values

# validation values
valid_names = valid_df['image_id'].values
valid_labels = valid_df['label'].values

# train generator
def train_generator():
    while True:
        for start in range(0, num_train, batch_size):
            x_batch = []
            y_batch = []

            end = min(start + batch_size, num_train-1)
            for img_path in range(start, end):
                img = Image.open(train_x[img_path])
                img = img.resize((img_width, img_height))
                #img_array = np.asarray(img, np.float32)
                img_array = np.array(img)
                #img_array = np.float32(img_array/255.0)
                #img_array = augment(img_array)
                x_batch.append(img_array)
                y_batch.append(train_y[img_path])

            yield (np.array(x_batch), np.array(y_batch))


# valid generator
def valid_generator():
    while True:
        for start in range(0, num_valid, batch_size):
            x_batch = []
            y_batch = []

            end = min(start + batch_size, num_valid-1)
            for img_path in range(start, end):
                img = Image.open(train_x[img_path])
                img = img.resize((img_width, img_height))
                #img_array = np.asarray(img, np.float32)
                img_array = np.array(img)
                #img_array = np.float32(img_array/255.0)
                #img_array = augment(img_array)
                x_batch.append(img_array)
                y_batch.append(valid_y[img_path])

            yield (np.array(x_batch), np.array(y_batch))

In [60]:
# callbacks
callbacks = [ReduceLROnPlateau(
                               monitor='val_loss', 
                               patience=2, 
                               verbose=1, 
                               factor=0.1, 
                               min_delta = 0.001, 
                               mode = 'min'
                              )]

In [66]:
# define pretrained model
pre_trained_model = EfficientNetB0(
                                   weights='imagenet', 
                                   include_top=False,
                                   input_shape=input_shape,
                                  )

#pre_trained_model.load_weights('efficientnet-b3_advprop_notop.h5')
#pre_trained_model.trainable = True

In [62]:
# add L2 regularization to pre-trained model
regularizer = tf.keras.regularizers.l2(l2_penalty)

for layer in pre_trained_model.layers:
    for attr in ['kernel_regularizer']:
        if hasattr(layer, attr):
          setattr(layer, attr, regularizer)
        
model_json = pre_trained_model.to_json()
pre_trained_model.save_weights('tmp_weights.h5')
pre_trained_model = tf.keras.models.model_from_json(model_json)
pre_trained_model.load_weights('tmp_weights.h5')

In [67]:
# define model
model = Sequential()
model.add(pre_trained_model)
model.add(GlobalAveragePooling2D())
#model.add(Dropout(dropout_rate))
model.add(Dense(5, activation="softmax"))

# compile model
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
              metrics = ['accuracy'])

# summarize model
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 6405      
Total params: 4,055,976
Trainable params: 4,013,953
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
# fit the model
history = model.fit(
    train_generator(),
    epochs= 10,
    steps_per_epoch= num_train // batch_size,
    validation_data= valid_generator(),
    validation_steps = num_valid // batch_size,
    #class_weight=class_weights,
    #callbacks=callbacks,
)   

Epoch 1/10
4204/4279 [============================>.] - ETA: 7s - loss: 0.8271 - accuracy: 0.7055

In [26]:
print(train_y[0])
print(train_y[1])
print(train_y[2])
print(train_y[3])

3
3
3
2


In [12]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Accuracy over epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.show()

NameError: name 'history' is not defined